```{contents}
```
## **Debate Pattern in LangGraph**

The **Debate pattern** in LangGraph is a **multi-agent reasoning architecture** where multiple LLM agents independently propose solutions, **critique each other**, and iteratively refine their arguments until a **consensus or final decision** is reached.
It is designed to reduce hallucination, improve reasoning reliability, and surface hidden assumptions.

---

### **1. Motivation**

Single-pass LLM answers suffer from:

| Limitation        | Impact             |
| ----------------- | ------------------ |
| Shallow reasoning | Missed edge cases  |
| Hallucinations    | Unverified claims  |
| Confirmation bias | One perspective    |
| Overconfidence    | No self-correction |

Debate introduces **structured disagreement** to improve solution quality.

---

### **2. Conceptual Workflow**

```
Problem
   ↓
Agent A → Proposal A
Agent B → Proposal B
Agent C → Proposal C
   ↓
Critique & Counter-Arguments
   ↓
Refinement Loop
   ↓
Consensus / Final Answer
```

---

### **3. Graph-Level Design**

### State Schema

```python
class DebateState(TypedDict):
    question: str
    arguments: list[str]
    critiques: list[str]
    round: int
    final_answer: str
```

---

### Nodes

| Node     | Responsibility             |
| -------- | -------------------------- |
| Propose  | Generate initial solutions |
| Critique | Evaluate others' arguments |
| Refine   | Improve proposals          |
| Decide   | Select final answer        |

---

### Control Flow

```
Propose → Critique → Refine → (loop until stable) → Decide → END
```

---

### **4. Minimal LangGraph Implementation**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict

class DebateState(TypedDict):
    question: str
    arguments: list[str]
    critiques: list[str]
    round: int
    final: str

def propose(state):
    args = state.get("arguments", [])
    args.append(f"Proposal round {state['round']}")
    return {"arguments": args}

def critique(state):
    crits = state.get("critiques", [])
    crits.append("Critique of last proposal")
    return {"critiques": crits}

def refine(state):
    return {"round": state["round"] + 1}

def decide(state):
    return {"final": "Best combined answer"}

builder = StateGraph(DebateState)

builder.add_node("propose", propose)
builder.add_node("critique", critique)
builder.add_node("refine", refine)
builder.add_node("decide", decide)

builder.set_entry_point("propose")
builder.add_edge("propose", "critique")
builder.add_edge("critique", "refine")

def route(state):
    return "decide" if state["round"] >= 3 else "propose"

builder.add_conditional_edges("refine", route, {
    "propose": "propose",
    "decide": "decide"
})

builder.add_edge("decide", END)

graph = builder.compile()
result = graph.invoke({"question": "Is LangGraph better than chains?", "round": 1, "arguments": [], "critiques": []})
```

---

### **5. Variants of Debate**

| Variant                  | Purpose                 |
| ------------------------ | ----------------------- |
| Two-Agent Debate         | Lightweight reasoning   |
| Multi-Agent Debate       | High-stakes decisions   |
| Expert Panel             | Role-specialized agents |
| Adversarial Debate       | Stress-test assumptions |
| Human-in-the-Loop Debate | Compliance & safety     |
| Judge Agent              | Neutral arbitration     |

---

### **6. When to Use Debate**

| Scenario                 | Benefit               |
| ------------------------ | --------------------- |
| Complex decision-making  | Improved correctness  |
| Research synthesis       | Reduced hallucination |
| Legal / medical analysis | Safer outputs         |
| Strategic planning       | Better risk modeling  |

---

### **7. Production Safeguards**

| Control          | Reason                 |
| ---------------- | ---------------------- |
| Max rounds       | Prevent infinite loops |
| Argument pruning | Control token cost     |
| Agent isolation  | Avoid leakage          |
| Audit logging    | Trace decisions        |
| Human approval   | Critical actions       |

---

### **8. Why Debate Works**

Debate approximates **collective intelligence**:

> Independent reasoning + structured conflict → superior solutions.


### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# ----------------------------
# 1. Define State
# ----------------------------

class DebateState(TypedDict):
    question: str
    arguments: list[str]
    critiques: list[str]
    round: int
    final: str

# ----------------------------
# 2. Define Nodes
# ----------------------------

def propose(state):
    args = state["arguments"]
    args.append(f"Proposal from agent: solution idea at round {state['round']}")
    return {"arguments": args}

def critique(state):
    crits = state["critiques"]
    crits.append(f"Critique: potential flaw found in round {state['round']}")
    return {"critiques": crits}

def refine(state):
    return {"round": state["round"] + 1}

def decide(state):
    combined = " | ".join(state["arguments"][-2:])
    return {"final": f"Final consensus based on debate → {combined}"}

# ----------------------------
# 3. Build Graph
# ----------------------------

builder = StateGraph(DebateState)

builder.add_node("propose", propose)
builder.add_node("critique", critique)
builder.add_node("refine", refine)
builder.add_node("decide", decide)

builder.set_entry_point("propose")
builder.add_edge("propose", "critique")
builder.add_edge("critique", "refine")

def router(state):
    return "decide" if state["round"] >= 3 else "propose"

builder.add_conditional_edges("refine", router, {
    "propose": "propose",
    "decide": "decide"
})

builder.add_edge("decide", END)

graph = builder.compile()

# ----------------------------
# 4. Run Debate
# ----------------------------

result = graph.invoke({
    "question": "Is LangGraph suitable for building autonomous agents?",
    "arguments": [],
    "critiques": [],
    "round": 1,
    "final": ""
})

print("Final Answer:\n", result["final"])
print("\nAll Arguments:\n", result["arguments"])
print("\nAll Critiques:\n", result["critiques"])


Final Answer:
 Final consensus based on debate → Proposal from agent: solution idea at round 1 | Proposal from agent: solution idea at round 2

All Arguments:
 ['Proposal from agent: solution idea at round 1', 'Proposal from agent: solution idea at round 2']

All Critiques:
 ['Critique: potential flaw found in round 1', 'Critique: potential flaw found in round 2']


| Dimension             | Debate                   | Negotiation                          |
| --------------------- | ------------------------ | ------------------------------------ |
| Primary Goal          | **Find the best answer** | **Reach acceptable agreement**       |
| Optimization Target   | **Correctness & truth**  | **Mutual satisfaction / compromise** |
| Nature of Interaction | Competitive + critical   | Cooperative + strategic              |
| Outcome Type          | One correct solution     | Trade-off solution                   |
